In [0]:
print("\n" + "="*60)
print("🏆 CRÉATION DE LA COUCHE GOLD - MODÈLE DIMENSIONNEL")
print("="*60)
from pyspark.sql import functions as F
from pyspark.sql import Window
import os
# ============================================================
# ÉTAPE 0 : RÉCUPÉRATION DES VARIABLES D'ENVIRONNEMENT
# ============================================================
print(f"\n🔧 Récupération des variables d'environnement...")
print("-" * 60)
storage_account_name = os.environ.get("AZURE_STORAGE_ACCOUNT_NAME")
storage_account_key = os.environ.get("AZURE_STORAGE_ACCOUNT_KEY")
if not storage_account_name or not storage_account_key:
    print("❌ Erreur : Variables d'environnement non trouvées")
    print("  ℹ️  Vérification des variables disponibles...")
    
    # Essayer d'autres noms possibles
    storage_account_name = os.environ.get("STORAGE_ACCOUNT_NAME")
    storage_account_key = os.environ.get("STORAGE_ACCOUNT_KEY")
    
    if storage_account_name and storage_account_key:
        print("✅ Variables trouvées avec noms alternatifs")
    else:
        raise ValueError("Variables AZURE_STORAGE_ACCOUNT_NAME et AZURE_STORAGE_ACCOUNT_KEY non définies")
print(f"✅ Storage Account : {storage_account_name}")
# Configuration Spark
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)
print("✅ Configuration Spark effectuée")
# ============================================================
# ÉTAPE 1 : CONFIGURATION DU SCHÉMA GOLD
# ============================================================
print(f"\n🔧 Configuration du schéma Gold...")
print("-" * 60)
# Utiliser le catalog hive_metastore
spark.sql("USE CATALOG hive_metastore")
# Créer le schéma gold s'il n'existe pas
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
spark.sql("USE SCHEMA gold")
print("✅ Schéma 'gold' créé/utilisé")
# Configuration Azure Data Lake
container_gold = "gold"
gold_base = f"abfss://{container_gold}@{storage_account_name}.dfs.core.windows.net/"
# Chemins pour chaque table
gold_dim_temps     = gold_base + "dim_temps/"
gold_dim_commune   = gold_base + "dim_commune/"
gold_dim_reseau    = gold_base + "dim_reseau/"
gold_dim_parametre = gold_base + "dim_parametre/"
gold_fact_prelev   = gold_base + "fact_prelevement/"
gold_fact_result   = gold_base + "fact_resultat/"
print(f"📂 Conteneur Gold : {container_gold}")
print(f"📂 Chemin de base : {gold_base}")

In [0]:
# ============================================================
# ÉTAPE 2 : LECTURE DES TABLES SILVER
# ============================================================
print(f"\n📖 Lecture des tables Silver...")
print("-" * 60)
plv = spark.table("eau_potable.silver_plv_clean_2")
res = spark.table("eau_potable.silver_result_clean_2")
nb_plv = plv.count()
nb_res = res.count()
print(f"✅ silver_plv_clean_2 : {nb_plv:,} lignes")
print(f"✅ silver_result_clean_2 : {nb_res:,} lignes")


In [0]:
# ============================================================
# ÉTAPE 3 : CRÉATION DES DIMENSIONS
# ============================================================
print(f"\n🎯 Création des dimensions...")
print("="*60)
# ---------- DIM_TEMPS ----------
print(f"\n📅 Dimension : dim_temps")
print("-" * 60)
dim_temps = (plv
    .select(F.col("dateprel").alias("date"))
    .where(F.col("date").isNotNull())
    .dropDuplicates()
    .withColumn("date_key", F.date_format("date", "yyyyMMdd").cast("int"))
    .withColumn("annee", F.year("date"))
    .withColumn("trimestre", F.quarter("date"))
    .withColumn("mois", F.month("date"))
    .withColumn("semaine", F.weekofyear("date"))
    .withColumn("jour", F.dayofmonth("date"))
    .withColumn("jour_semaine", F.date_format("date", "EEEE"))
    .withColumn("mois_nom", F.date_format("date", "MMMM"))
)
nb_dim_temps = dim_temps.count()
print(f"  ✅ Dates distinctes : {nb_dim_temps:,}")
# Sauvegarder
(dim_temps
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_dim_temps)
 .saveAsTable("hive_metastore.gold.dim_temps"))
print(f"  ✅ Table dim_temps créée")
# ---------- DIM_COMMUNE ----------
print(f"\n🏘️  Dimension : dim_commune")
print("-" * 60)
dim_commune = (plv
    .select(
        F.col("inseecommuneprinc").alias("code_insee"),
        F.col("nomcommuneprinc").alias("nom_commune"),
        F.col("cddept").alias("code_dept")
    )
    .where(F.col("code_insee").isNotNull())
    .dropDuplicates(["code_insee", "nom_commune", "code_dept"])
)
# Créer une clé surrogate stable (hash)
dim_commune = dim_commune.withColumn(
    "commune_sk",
    F.sha2(F.concat_ws("||", 
        F.col("code_insee"), 
        F.col("nom_commune"), 
        F.col("code_dept")
    ), 256)
)
nb_dim_commune = dim_commune.count()
print(f"  ✅ Communes distinctes : {nb_dim_commune:,}")
# Sauvegarder
(dim_commune
 .select("commune_sk", "code_insee", "nom_commune", "code_dept")
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_dim_commune)
 .saveAsTable("hive_metastore.gold.dim_commune"))
print(f"  ✅ Table dim_commune créée")
# ---------- DIM_RESEAU ----------
print(f"\n🔗 Dimension : dim_reseau")
print("-" * 60)
dim_reseau = (plv
    .select(
        F.col("cdreseau").alias("code_reseau"),
        F.col("cdreseauamont").alias("code_reseau_amont"),
        F.col("nomreseauamont").alias("nom_reseau_amont")
    )
    .where(F.col("code_reseau").isNotNull())
    .dropDuplicates(["code_reseau", "code_reseau_amont", "nom_reseau_amont"])
)
# Créer une clé surrogate
dim_reseau = dim_reseau.withColumn(
    "reseau_sk",
    F.sha2(F.concat_ws("||",
        F.col("code_reseau"),
        F.coalesce(F.col("code_reseau_amont"), F.lit("")),
        F.coalesce(F.col("nom_reseau_amont"), F.lit(""))
    ), 256)
)
nb_dim_reseau = dim_reseau.count()
print(f"  ✅ Réseaux distincts : {nb_dim_reseau:,}")
# Sauvegarder
(dim_reseau
 .select("reseau_sk", "code_reseau", "code_reseau_amont", "nom_reseau_amont")
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_dim_reseau)
 .saveAsTable("hive_metastore.gold.dim_reseau"))
print(f"  ✅ Table dim_reseau créée")
# ---------- DIM_PARAMETRE ----------
print(f"\n🧪 Dimension : dim_parametre")
print("-" * 60)
dim_parametre = (res
    .select(
        F.col("cdparametre").alias("code_parametre"),
        F.col("cdparametresiseeaux").alias("code_parametre_sise"),
        F.col("libmajparametre").alias("libelle_parametre"),
        F.col("libwebparametre").alias("libelle_web"),
        F.col("categorie_parametre"),
        F.col("sous_categorie"),
        F.col("cdunitereference").alias("code_unite"),
        F.col("cdunitereferencesiseeaux").alias("code_unite_sise"),
        F.col("limitequal").alias("limite_qualite"),
        F.col("refqual").alias("reference_qualite"),
        F.col("qualitparam").alias("type_parametre")
    )
    .where(F.col("code_parametre").isNotNull())
    .dropDuplicates()
)
# Créer clé surrogate
dim_parametre = dim_parametre.withColumn(
    "parametre_sk",
    F.sha2(F.concat_ws("||",
        F.coalesce(F.col("code_parametre"), F.lit("")),
        F.coalesce(F.col("code_unite"), F.lit("")),
        F.coalesce(F.col("libelle_parametre"), F.lit(""))
    ), 256)
)
nb_dim_parametre = dim_parametre.count()
print(f"  ✅ Paramètres distincts : {nb_dim_parametre:,}")
# Sauvegarder
(dim_parametre
 .select("parametre_sk", "code_parametre", "code_parametre_sise", 
         "libelle_parametre", "libelle_web", "categorie_parametre", 
         "sous_categorie", "code_unite", "code_unite_sise", 
         "limite_qualite", "reference_qualite", "type_parametre")
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_dim_parametre)
 .saveAsTable("hive_metastore.gold.dim_parametre"))
print(f"  ✅ Table dim_parametre créée")


In [0]:
# ============================================================
# ÉTAPE 4 : CRÉATION DES TABLES DE FAITS
# ============================================================
print(f"\n📊 Création des tables de faits...")
print("="*60)
# Charger les dimensions avec leurs clés
dim_temps_keyed = spark.table("hive_metastore.gold.dim_temps").select(
    F.col("date").alias("d_date"),
    F.col("date_key")
)
dim_commune_keyed = spark.table("hive_metastore.gold.dim_commune").select(
    "commune_sk", "code_insee", "code_dept"
)
dim_reseau_keyed = spark.table("hive_metastore.gold.dim_reseau").select(
    "reseau_sk", "code_reseau"
)
dim_parametre_keyed = spark.table("hive_metastore.gold.dim_parametre").select(
    "parametre_sk", "code_parametre", "code_unite"
)

In [0]:
# ---------- FACT_PRELEVEMENT ----------
print(f"\n💧 Fait : fact_prelevement")
print("-" * 60)
fact_prelev = (plv
    .join(dim_commune_keyed, 
          (plv["inseecommuneprinc"] == dim_commune_keyed["code_insee"]) &
          (plv["cddept"] == dim_commune_keyed["code_dept"]), 
          "left")
    .join(dim_reseau_keyed, 
          plv["cdreseau"] == dim_reseau_keyed["code_reseau"], 
          "left")
    .join(dim_temps_keyed, 
          plv["dateprel"] == dim_temps_keyed["d_date"], 
          "left")
    .select(
        # Clés de dimension
        F.col("commune_sk"),
        F.col("reseau_sk"),
        F.col("date_key"),
        
        # Clés naturelles
        F.col("referenceprel").alias("reference_prelevement"),
        F.col("cddept").alias("code_dept"),
        F.col("annee"),
        
        # Attributs du prélèvement
        F.col("dateprel").alias("date_prelevement"),
        F.col("heure_prel").alias("heure_prelevement"),
        F.col("pourcentdebit").alias("pourcentage_debit"),
        F.col("conclusionprel").alias("conclusion"),
        
        # Conformités
        F.col("plvconformitebacterio").alias("conformite_bacterio"),
        F.col("plvconformitechimique").alias("conformite_chimique"),
        F.col("plvconformitereferencebact").alias("conformite_ref_bacterio"),
        
        # Enrichissement temporel
        F.col("mois_prel"),
        F.col("trimestre_prel"),
        F.col("semestre_prel"),
        F.col("jour_semaine")
    )
    .dropDuplicates(["reference_prelevement"])
)
nb_fact_prelev = fact_prelev.count()
print(f"  ✅ Prélèvements : {nb_fact_prelev:,} lignes")
# Sauvegarder
(fact_prelev
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_fact_prelev)
 .saveAsTable("hive_metastore.gold.fact_prelevement"))
print(f"  ✅ Table fact_prelevement créée")
# ---------- FACT_RESULTAT ----------
print(f"\n🔬 Fait : fact_resultat")
print("-" * 60)
# Jointure res + plv pour récupérer les infos géographiques et temporelles
fact_result = (res.alias("r")
    .join(plv.select(
            "referenceprel", "inseecommuneprinc", "cddept", 
            "cdreseau", "dateprel", "annee"
        ).alias("p"),
        F.col("r.referenceprel") == F.col("p.referenceprel"), 
        "left")
    .join(dim_commune_keyed.alias("c"),
          (F.col("p.inseecommuneprinc") == F.col("c.code_insee")) &
          (F.col("p.cddept") == F.col("c.code_dept")),
          "left")
    .join(dim_reseau_keyed.alias("re"),
          F.col("p.cdreseau") == F.col("re.code_reseau"),
          "left")
    .join(dim_temps_keyed.alias("t"),
          F.col("p.dateprel") == F.col("t.d_date"),
          "left")
    .join(dim_parametre_keyed.alias("dp"),
          (F.col("r.cdparametre") == F.col("dp.code_parametre")) &
          ((F.col("r.cdunitereference") == F.col("dp.code_unite")) |
           F.col("dp.code_unite").isNull()),
          "left")
    .select(
        # Clés de dimension
        F.col("c.commune_sk"),
        F.col("re.reseau_sk"),
        F.col("t.date_key"),
        F.col("dp.parametre_sk"),
        
        # Clés naturelles
        F.col("r.referenceprel").alias("reference_prelevement"),
        F.col("r.cddept").alias("code_dept"),
        F.col("r.annee"),
        
        # Valeurs mesurées
        F.col("r.valtraduite").alias("valeur_numerique"),
        F.col("r.rqana").alias("resultat_brut"),
        
        # Attributs de l'analyse
        F.col("r.qualitparam").alias("type_parametre"),
        F.col("r.insituana").alias("lieu_analyse"),
        F.col("r.cdunitereference").alias("unite_mesure"),
        F.col("r.limitequal").alias("limite_qualite"),
        F.col("r.refqual").alias("reference_qualite"),
        F.col("r.casparam").alias("cas_parametre"),
        F.col("r.referenceanl").alias("reference_analyse")
    )
)
nb_fact_result = fact_result.count()
print(f"  ✅ Résultats : {nb_fact_result:,} lignes")
# Sauvegarder
(fact_result
 .write.format("delta").mode("overwrite").option("overwriteSchema", "true")
 .option("path", gold_fact_result)
 .saveAsTable("hive_metastore.gold.fact_resultat"))
print(f"  ✅ Table fact_resultat créée")


In [0]:
%sql
use catalog `hive_metastore`; select * from `gold`.`dim_commune` limit 100;

In [0]:
%sql
-- ============================================================
-- 📊 REQUÊTES GOLD - AVRIL 2024 - CODE INSEE 59009
-- ============================================================

-- ========================================
-- 1️⃣ VUE D'ENSEMBLE : Prélèvements avril 2024 pour commune 59009
-- ========================================

USE CATALOG `hive_metastore`;

SELECT 
    -- Informations commune
    c.code_insee,
    c.nom_commune,
    c.code_dept,
    
    -- Informations temporelles
    t.date,
    t.annee,
    t.mois,
    t.jour_semaine,
    
    -- Informations réseau
    r.code_reseau,
    r.nom_reseau_amont,
    
    -- Détails prélèvement
    fp.reference_prelevement,
    fp.heure_prelevement,
    fp.pourcentage_debit,
    fp.conclusion,
    
    -- Conformités
    fp.conformite_bacterio,
    fp.conformite_chimique,
    fp.conformite_ref_bacterio
    
FROM `gold`.`fact_prelevement` fp

-- Jointure avec dimensions
LEFT JOIN `gold`.`dim_commune` c ON fp.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_reseau` r ON fp.reseau_sk = r.reseau_sk
LEFT JOIN `gold`.`dim_temps` t ON fp.date_key = t.date_key

-- Filtres
WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4

ORDER BY t.date DESC, fp.heure_prelevement DESC

LIMIT 100;

In [0]:
%sql
-- ========================================
-- 2️⃣ RÉSULTATS D'ANALYSES : Avril 2024 - Commune 59009
-- ========================================

SELECT 
    -- Informations commune
    c.code_insee,
    c.nom_commune,
    
    -- Informations temporelles
    t.date,
    t.annee,
    t.mois,
    
    -- Informations prélèvement
    fr.reference_prelevement,
    
    -- Informations paramètre
    p.code_parametre,
    p.libelle_parametre,
    p.categorie_parametre,
    p.sous_categorie,
    p.code_unite,
    
    -- Résultats
    fr.valeur_numerique,
    fr.resultat_brut,
    fr.type_parametre,
    fr.lieu_analyse,
    
    -- Limites réglementaires
    p.limite_qualite,
    p.reference_qualite
    
FROM `gold`.`fact_resultat` fr

-- Jointures
LEFT JOIN `gold`.`dim_commune` c ON fr.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_temps` t ON fr.date_key = t.date_key
LEFT JOIN `gold`.`dim_parametre` p ON fr.parametre_sk = p.parametre_sk

-- Filtres
WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4

ORDER BY t.date DESC, p.categorie_parametre, p.libelle_parametre

LIMIT 100;

In [0]:
%sql
-- ========================================
-- 3️⃣ RÉSUMÉ PAR CATÉGORIE : Avril 2024 - Commune 59009
-- ========================================

SELECT 
    -- Catégorie de paramètre
    p.categorie_parametre,
    
    -- Statistiques
    COUNT(DISTINCT fr.reference_prelevement) as nb_prelevements,
    COUNT(*) as nb_analyses,
    COUNT(DISTINCT p.code_parametre) as nb_parametres_distincts,
    
    -- Valeurs moyennes (pour paramètres numériques)
    ROUND(AVG(fr.valeur_numerique), 2) as valeur_moyenne,
    ROUND(MIN(fr.valeur_numerique), 2) as valeur_min,
    ROUND(MAX(fr.valeur_numerique), 2) as valeur_max
    
FROM `gold`.`fact_resultat` fr

LEFT JOIN `gold`.`dim_commune` c ON fr.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_temps` t ON fr.date_key = t.date_key
LEFT JOIN `gold`.`dim_parametre` p ON fr.parametre_sk = p.parametre_sk

WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4
  AND fr.valeur_numerique IS NOT NULL

GROUP BY p.categorie_parametre
ORDER BY nb_analyses DESC;

In [0]:
%sql
-- ========================================
-- 4️⃣ TOP 10 PARAMÈTRES : Avril 2024 - Commune 59009
-- ========================================

SELECT 
    -- Paramètre
    p.code_parametre,
    p.libelle_parametre,
    p.categorie_parametre,
    p.code_unite,
    
    -- Statistiques
    COUNT(*) as nb_mesures,
    ROUND(AVG(fr.valeur_numerique), 3) as valeur_moyenne,
    ROUND(MIN(fr.valeur_numerique), 3) as valeur_min,
    ROUND(MAX(fr.valeur_numerique), 3) as valeur_max,
    
    -- Limite réglementaire
    p.limite_qualite
    
FROM `gold`.`fact_resultat` fr

LEFT JOIN `gold`.`dim_commune` c ON fr.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_temps` t ON fr.date_key = t.date_key
LEFT JOIN `gold`.`dim_parametre` p ON fr.parametre_sk = p.parametre_sk

WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4
  AND fr.valeur_numerique IS NOT NULL

GROUP BY 
    p.code_parametre,
    p.libelle_parametre,
    p.categorie_parametre,
    p.code_unite,
    p.limite_qualite

ORDER BY nb_mesures DESC
LIMIT 10;

In [0]:
%sql
-- ========================================
-- 5️⃣ CONFORMITÉ : Avril 2024 - Commune 59009
-- ========================================

SELECT 
    c.nom_commune,
    t.date,
    fp.reference_prelevement,
    
    -- Conformités
    fp.conformite_bacterio,
    fp.conformite_chimique,
    fp.conformite_ref_bacterio,
    
    -- Conclusion globale
    fp.conclusion,
    
    -- Réseau
    r.nom_reseau_amont
    
FROM `gold`.`fact_prelevement` fp

LEFT JOIN `gold`.`dim_commune` c ON fp.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_temps` t ON fp.date_key = t.date_key
LEFT JOIN `gold`.`dim_reseau` r ON fp.reseau_sk = r.reseau_sk

WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4

ORDER BY t.date DESC;

In [0]:
%sql
-- ========================================
-- 7️⃣ ÉVOLUTION TEMPORELLE : Avril 2024 - Commune 59009
-- ========================================

SELECT 
    t.date,
    t.jour_semaine,
    COUNT(DISTINCT fp.reference_prelevement) as nb_prelevements,
    
    -- Conformités agrégées
    SUM(CASE WHEN fp.conformite_bacterio = 'C' THEN 1 ELSE 0 END) as conformes_bacterio,
    SUM(CASE WHEN fp.conformite_chimique = 'C' THEN 1 ELSE 0 END) as conformes_chimique,
    
    -- Taux de conformité
    ROUND(
        SUM(CASE WHEN fp.conformite_bacterio = 'C' THEN 1 ELSE 0 END) * 100.0 / 
        COUNT(DISTINCT fp.reference_prelevement), 
        1
    ) as taux_conformite_bacterio_pct
    
FROM `gold`.`fact_prelevement` fp

LEFT JOIN `gold`.`dim_commune` c ON fp.commune_sk = c.commune_sk
LEFT JOIN `gold`.`dim_temps` t ON fp.date_key = t.date_key

WHERE c.code_insee = '59009'
  AND t.annee = 2024
  AND t.mois = 4

GROUP BY t.date, t.jour_semaine
ORDER BY t.date;